In [22]:
# get valid files from multiple directories
import os

def get_filename(prefix, directory):
    filenames = []
    if prefix == 'fast_flux':
        prefix_dir = ['64_32_8_70_TIME_2020-03-01-22-30-00',
        '64_32_8_69_TIME_2020-03-01-22-32-00',
        '5_79_68_109_TIME_2020-03-05-21-20-07',
        '5_79_68_108_TIME_2020-03-01-22-05-04',
        '46_166_182_113_TIME_2020-03-05-21-19-39',
        '46_166_182_109_TIME_2020-03-01-20-33-31',
        '37_48_65_151_TIME_2020-03-05-21-20-19',
        '37_48_65_150_TIME_2020-03-01-21-25-11',
        '37_48_65_149_TIME_2020-03-01-20-48-28',
        '37_48_65_148_TIME_2020-03-01-21-14-04',
        '207_244_67_215_TIME_2020-03-01-20-49-33',
        '207_244_67_214_TIME_2020-03-05-21-20-32',
        '207_244_67_214_TIME_2020-03-05-21-19-55',
        '207_244_67_139_TIME_2020-03-01-22-28-12',
        '207_244_67_138_TIME_2020-03-01-20-36-51',
        '162_210_195_123_TIME_2020-03-01-21-07-43',
        '162_210_195_122_TIME_2020-03-01-21-31-36']

        domains = set([])
        for prefix in prefix_dir:
            directory = '/Users/zc/Documents/TwitterBotProject/malicious_domain/{}/'.format(prefix)
            for filename in os.listdir(directory):
                    if filename not in domains:
                        domains.add(filename)
                        filenames.append('{}/{}/trace.json'.format(directory, filename))
        print(len(filenames))
        print(len(set(filenames)))
    
    else:
        for filename in os.listdir(directory):
            # load trace.json if it exists
            filenames.append('{}/{}/trace.json'.format(directory, filename))
#             if os.path.isfile('{}/{}/trace.json'.format(directory, filename)):
#                 trace = json.load(open('{}/{}/trace.json'.format(directory, filename), 'r'))
    return filenames


In [23]:
# visualize domain redirection chain with d3js

import os
import json
from pprint import pprint
from urllib.parse import urlparse
import util

def get_redirection_chain(document, edge_type):
    has_frontend_redirect = False
    previous_url = urlparse(document['url']).netloc
    # 1. init sub_redirect_chain with the first level URL
    sub_redirect_chain = [previous_url]
    
    # we need a map of url --> ip
    url_ip_map = {}
    for entry in document["network_communication"]:
        # check if remoteIPAddress points to NYU IP (no such case find)
        if entry['remoteIPAddress'] == '216.165.95.140':
            pprint(entry)
        normalized_url = util.get_normalized_domain(entry['url'])
        url_ip_map[normalized_url] = entry['remoteIPAddress']
        
    for index, chain in enumerate(document["redirection_chain"]):
        method = chain['method']
        
        if method == 'Page.frameScheduledNavigation':
            url_domain = urlparse(chain['url']).netloc
            if url_domain != previous_url:
                edge_type[(sub_redirect_chain[-1], url_domain)] = 'client_side'
                sub_redirect_chain.append(url_domain)
                has_frontend_redirect = True
            previous_url = urlparse(chain['url']).netloc

        if method == 'Network.requestWillBeSent':
            url_from = urlparse(chain['from_url']).netloc
            url_to = urlparse(chain['to_url']).netloc
            if url_from == previous_url and url_from != url_to:
                edge_type[(sub_redirect_chain[-1], url_to)] = 'server_side'
                sub_redirect_chain.append(url_to)
                previous_url = url_to
    
    return has_frontend_redirect, url_ip_map, sub_redirect_chain, document["network_communication"]


# dbsafe.net
# 64.32.8.67

# prefix = '207_244_67_139_TIME_2020-03-01-22-28-12'
# prefix = '46_166_182_113_TIME_2020-03-05-21-19-39'
# prefix = '181_214_86_147_TIME_2020-03-02-23-41-32'
# prefix = '64_32_8_67_TIME_2020-03-01-21-09-38'   # current choice of fast_flux
# prefix = '64_32_8_68_TIME_2020-03-01-22-26-19'
# prefix = 'nycpost_pro'
# prefix = '23_82_12_32_TIME_2020-03-25-21-40-52'
# prefix = 'cnnews3_com'
# prefix = '72_52_179_174_TIME_2020-03-25-21-43-18'
prefix = 'fast_flux'
directory = '/Users/zc/Documents/TwitterBotProject/malicious_domain/{}/'.format(prefix)
graph_type = 'domain'  # domain | ip

from collections import defaultdict
nodes = {}
links = defaultdict(int)
counter = 0
no_ip = 0
no_redirect = 0
has_redirect = 0
last_domain = set([])
first_domain = set([])
temp_count = 0
valid_first_ips = defaultdict(list)

filenames = get_filename(prefix, directory)

# for filename in os.listdir(directory):
#     # load trace.json if it exists
#     if os.path.isfile('{}/{}/trace.json'.format(directory, filename)):
#         trace = json.load(open('{}/{}/trace.json'.format(directory, filename), 'r'))

for filename in filenames:
    if os.path.isfile(filename):
        trace = json.load(open(filename, 'r'))
        if any([r['is_redirect'] for r in trace['record']]):
            final_chain = []
            if len(trace['record']) >= 1 and filename != 'redmenhockey_com':
#                 print(filename)
                domain_ip = {}
                for r in trace['record']:
                    _, url_ip_map, res, net_comm = get_redirection_chain(r['record'], {})
                    if len(final_chain) == 1 and final_chain[0] == res[0]:
                        final_chain = res
                        domain_ip = url_ip_map
                    else:
                        final_chain += res
                        domain_ip.update(url_ip_map)
                        
                # skip non-redirection chains
                final_chain = [url.replace('www.', '') for url in final_chain]
                if len(final_chain) < 2:
                    continue
                if final_chain[-1] in ['hugedomains.com', 'domainmarket.com']:
                    no_ip += 1
                    continue
                if len(final_chain) == 2 and final_chain[-1] == final_chain[-2]:
                    no_redirect += 1
                    continue
                
#                 if not ('client_server_location' in trace and 'USA' in trace['client_server_location']):
#                     continue
#                 valid_first_ips.append(domain_ip.get(final_chain[0].replace('www.', ''), None))
                
                if prefix == 'cnnews3_com':
                    if domain_ip.get(final_chain[0], None) != '103.224.182.207':
                        print('Different IP address detected, continue next')
                        continue
                        
                ##### translate domains to IPs ###############
                if graph_type == 'ip':
                    new_final_chain = []
                    pass_url = False
                    for url in final_chain:
                        if url not in domain_ip:
                            print("NO URL: ", url)
                            new_final_chain.append(str(temp_count))
                            temp_count += 1
                            pass_url = True
                        else:
                            new_final_chain.append(domain_ip[url])

                    if pass_url:
                        continue
                    final_chain = new_final_chain
            
#                 final_chain = [domain_ip[url.replace('www.', '')] for url in final_chain]
                #############################################
                
                # reformat final_chain to remove cycles
                previous_url = defaultdict(int)
                new_final_chain = []
                for url in final_chain:
                    num = previous_url[url]
                    if num == 0:
                        new_final_chain.append(url)
                    else:
                        new_final_chain.append('{}_LOOP_{}'.format(url, num))
                    previous_url[url] += 1
                
                # add node, edge
                final_chain = new_final_chain

                if len(final_chain) != len(set(final_chain)):
                    print('duplicate detected!')
                
                if 'joblift.com' in final_chain:
                    continue
                
                #### merge nodes names if necessary
                if prefix == 'cnnews3_com':
                    merge_dic = {'park.above.com': 'above', 
                             'bidr.trellian.com': 'trellian', 
#                              'clk.rtpdn11.com': 'rtpdn11',
#                              'ww2.siteplug.com': 'siteplug',
#                              'usa.jared-don.com': 'usa',
#                              'usd.jared-don.com': 'usb',
                            }

                    if '.ca' in final_chain[-1] or '.be' in final_chain[-1] or '.de' in final_chain[-1] or '.icu' in final_chain[-1]:
                        if '.icu' in final_chain[-1]:
                            print(final_chain[-1])
                        continue
                        
                    if len(set(final_chain).intersection(set(['avast-2015.en.softonic.com', 'tq.adventurefeeds.com', 'secure.clicktrkservices.com']))) > 0:
                        continue
                    
                    # TODO: maybe we should also filter out ICU
                    
                    if final_chain[-1] in set(['jcpenney.com', 'livecareer.7eer.net', 'prizewing.com', 'finder.com', 'browsingsecurityhub.com', 'nissanusa.com', 'cbp.gov', 'powercontor.com', 'stamps.com', 'multimilltrackz2.com', 'rd.bizrate.com', 'the-house.com', 'horseloverz.com', 'query.pureleads.com', 'justhost.com', 'bestsportsbetting.com_LOOP_1', 'burpee.com', 'parachutehome.com', 'frontgate.com', 'uneaskie.site', 'adultfriendfinder.com', 'groupon.com', 'intelius.com', 'made.com', 'apple-fr.productoftheyeargifts.xyz', 'honestloans.net', 'fabric.com', 'superstar.com', 'walletwise.us', 'usflights.sale', 'best.aliexpress.com', 'clk.rtpdn11.com', 'hautelook.com', 'psychicpromise.com', 'shein.com', 'greatcloudworks.com', 'google-en.greatproductdeals.site', 'resume-now.com', 'samsung.com', 'amazon.com', 'fr.stop-bot.com', 'macys.com', 'bilde3.b-cdn.net', 'reimageplus.com', 'freeshipping.com', 'worldoftanks.ru', 'rates.savingsaccounts.com', 'websitesecuritygroup.com', 'store.hp.com']):
                        continue

                    if final_chain[0] == '.'.join(final_chain[1].split('.')[1:]):
                        final_chain.pop(1) 
                                            
                    if final_chain[1] in merge_dic:
                        first_domain.add(final_chain[0])
                        valid_first_ips['tier_1'].append((final_chain[0], domain_ip.get(final_chain[0], None)))
                        final_chain[0] = 'PHISHING_' + merge_dic[final_chain[1]]
                    else:
                        continue
                    
                    # add remaining ip addresses
                    valid_first_ips['tier_3'].append((final_chain[-1], domain_ip.get(final_chain[-1], None)))
                    if len(final_chain) > 2:
                        valid_first_ips['tier_2'].append((final_chain[1], domain_ip.get(final_chain[1], None)))
                    
                    if 'softonic.com' in final_chain[-1]:
                        final_chain[-1] = 'ADWARE_1'
                
                if prefix == '64_32_8_67_TIME_2020-03-01-21-09-38':
                    merge_dic = {'usa.alwilda-ram.com': 'usa', 
                             'usd.alwilda-ram.com': 'usb', 
                             'dprtb.com': 'dprtb',
#                              'ww2.siteplug.com': 'siteplug',
                             'usa.jared-don.com': 'usa',
                             'usd.jared-don.com': 'usb',
                            }
                    if len(set(['noclick.connexity.com', 'affsafe.space', 'tq.adventurefeeds.com',
                                'track.offerstracks.co', 'www.adzuna.ca', 'atnpx.com', 'seates-clable.com',
                                'iqoption.com', 'compare-insurances.com', 'web.countertrck.com',
                                'www.topusajobs.com', 'lifeimpressions.net', 'www.linkedin.com',
                                'www.top5ficktreffen.de', 'affbank.com', 'www.top5sitederencontre.fr',
                                'uneaskie.com', 'coinrivet.com', 'worldoftanks.ru', 'bilde3.b-cdn.net',
                                'thefuncoolstuff.com', 'qualmode.website']).intersection(set(final_chain))) > 0:
                        continue
                    if final_chain[0] == '.'.join(final_chain[1].split('.')[1:]):
                        final_chain.pop(1) 
                    
                    valid_first_ips['tier_1'].append((final_chain[0], domain_ip.get(final_chain[0], None)))
                    valid_first_ips['tier_3'].append((final_chain[-1], domain_ip.get(final_chain[-1], None)))
                    if len(final_chain) > 2:
                        valid_first_ips['tier_2'].append((final_chain[1], domain_ip.get(final_chain[1], None)))
                        
                    if 'softonic.com' in final_chain[-1]:
                        final_chain[-1] = 'ADWARE_1_Aggreagte'
                    if '.icu' in final_chain[-1]:
                        final_chain[-1] = 'ADWARE_2_Aggreagte'                
                    if final_chain[1] in merge_dic:
                        final_chain[0] = 'PHISHING_' + merge_dic[final_chain[1]]
                    else:
                        continue
                
                if prefix == 'fast_flux': 
                    merge_dic = {'usd.jared-don.com': 'usd.jared-don',
                    'usa.jared-don.com': 'usa.jared-don',
                    'dprtb.com': 'dprtb',
                    'usd.alwilda-ram.com': 'usd.alwilda-ram',
                    'usa.alwilda-ram.com': 'usa.alwilda-ram',
                    'click.expmediadirect.com': 'click',
                    'usd.eadgar-the.com': 'usd.eadgar-the',
                    'usa.eadgar-the.com': 'usa.eadgar-the'}
#                     'api.quotes.com': 'api.quotes',
#                     'clicks.torromi.com': 'clicks.torromi',
#                     'sarah.tncrun.net': 'sarah.tncrun',
#                     '4179.roberat.com': '4179.roberat',
#                     'ww2.siteplug.com': 'ww2.siteplug',
#                     'iyfsearch.com': 'iyfsearch',
#                     'query.pureleads.com': 'query.pureleads',
#                     'directnavbt.com': 'directnavbt',
#                     'c.channeltraffic.net': 'channeltraffic'}
#                     'survey-smiles.com': 'survey-smiles',
#                     'clk.rtpdn11.com': 'clk.rtpdn11',
#                     'xml.onwardclick.com': 'onwardclick'
                    
                    if set(final_chain).intersection(set(['jobdiagnosis.com', 'rd.purejobalerts.com_LOOP_1'])):
                        continue
                    
                    if set(final_chain).intersection(set(['jobleads.com', 'macpaw.com', 'install-nowflash-fast.com', 'shipt.com', 'installflash-watch-now.com', 'kbb.com', 'fireplayersoftware.com', 'halting.top', 'install-newflash-fast.com', 'browsingsecurityhub.com', 'getflash-now-fast.com', 'click.appcast.io', 'moneymaster.guru', 'chrome.google.com', 'neuvoo.com', 'purejobalerts.com', 'signup.finddreamjobs.com', 'shoppers.instacart.com', 'us.tideri.com', 'auskunft.de', 'allthetopbananas.com', 'play-freeflash-fast.com', 'get-flash-free-now.com', 'getflash-free-now.com', 'tours-78-94.wellhello.com', 'installflash-upgrade-now.com', 'cinematicfanatic.com', 'yahoo.com_LOOP_1', 'install-freeflash-fast.com', 'linkedin.com', 'e-recruit.sap.lionbridge.com', 'installflash-download-now.com', 'ehscareers.com', 'internationalcupid.com', 'drive4total.com', 'feed.int.jobble.com', 'cooing.top', 'canadianlisted.com', 'designkiki.com', 'websitesecuritygroup.com', 'vnxhgsxw.novelepoxy.com', 'usflights.sale', 'livecareer.7eer.net', 'genesiscareers.jobs', 'shop.nordstrom.com', 'fubo.tv', 'amazonhvh.thejobnetwork.com', 'adzuna.com', 'iqoption.com', 'analysisable.com', 'jackpots.exposed', 'totaljobs.com', 'info.jobble.com', 'worldoftanks.ru', 'rd.purejobalerts.com', 'hirelifescience.com', 'njump.xmobistein.com', 'shopperdeal.net', 'google.com', 'star-click.net', 'maytruckingdrivers.com', 'click.appcast.io_LOOP_1', 'powercontor.com', 'go.moartraffic.com', 'yourdocstopdf.com', 'bing.com', 'juju.com', 'eta.ueep.com', 'redirecting.mobile-redirect.com', 'rd.bizrate.com', 'pawskie.com', 'robogarden.io', 'cornerstone.recsolu.com', 'cbp.gov', 'justforguys.com', 'resignation.top', 'p7campaign.com', 'couponclubapp.co', 'affsafe.space', 'pkiniswve.club', 'esmeralda-free-psychic.com', 'ttrroniqus.xyz', 'doordash.com', 'nursecareerboard.com', 'magnumdrivers.com', 'questionnaire3-dish.icims.com', 'ozarktruckingjobs.com', 'c.adsco.re', 'usajobs.gov', 'track.leonbetvouum.com', 'fehu.fashion', 'yltenim.com', 'app2.mackeeperaffiliates.com', 'www3.imaginativerecordzz.xyz', 'port38.ihdio.com', 'joblookup.com', 'www2.originalmechanicszz.xyz', 'careers-dignityhealth.icims.com', '24newsoft.mainsourceoffreeupdates.best', 'jobs.cascades.com', 'privacysearching.com', 'drive4decker.com', 'beyourxfriend.com', 'store.uprightpose.com', 'myvancl.com', 'hulu.com_LOOP_1', 'buzzihub.ck.page', 'infopicked.com', 'metrowhiz.com', 'kiwi.com', 'jobleads.de', 'track.underworkingunwaverable.com', 'cpvcibermarketing.com', 'audiobooks.com', 'aditmedia.g2afse.com', 'datacompax.com', 'us.norton.com', 'seasoned.co', 'iloveinterracial.com', 'gearbest.com', 'themeforest.net', 'macys.com', 'quickbooks.intuit.com', 'autoquoteclub.com', 'american-consumer-panels.com', 'uber.com', 'wendys.taleo.net', 'atnpx.com', 'secure3.hilton.com', 'interscot.net_LOOP_1', 'shoplikeher.com_LOOP_1', 'install.livepdfconverter.com', 'adorama.com', 'lyst.com', 'freeshipping.com', 'promo.myflightcenter.net', 'lp.powerapp.download', 'childalertporn10.azurewebsites.net', 'lightbulbs.com', 'launchrenewedheavilyfreeware.com', 'glassdoor.com_LOOP_1', 'jobsintrucks.com', 'gettruckerjobs.com', 'disneyplus.com', 'evenghistougher.pro', 'truckdrivingjobs.com', 'careers.hireology.com', 'wolve.pro', 'devilwears.com', 'driveforhubgrouptrucking.com', '4qvci.leadnote.me', 'careers.hcahealthcare.com', 'coupons.offerhub.net', 'ads.midwayusa.com', 'bloomingdales.com', 'j1bai.leadnote.me', 'freeonlinegames.win', 'saatva.com_LOOP_1', 'myfirstwebnet.com', 'aeropostale.com', 'allbirds.com', 'go.rover.com', 'trc.adtechnacity.com', 'homepros.zone', 'collegerecruiter.com', 'poptropica.com', 'joinmclane.com', 'findatruckerjob.com', 'www6.imaginativejournalzz.xyz', 'mybestsecureus.com', 'applytracking.com', 'onlinenewsletters.net', 'rustoleum-openhire.silkroad.com', 'drive4hogan.com', 'nextcareernow.com', 'finder.com', 'uk.stop-bot.com', 'sciencenaturalsupplements.com', 'fr.stop-bot.com', 'affbank.com', 'antifraud-system.com', 'secured.mobile-redirect.com', 'install.convertmyvid.com', 'driverfixersoftware.com'])):
                        continue
                    
                    if set(final_chain).intersection(set(['ojgvxphcs.queenvalery.com', 'pocus.vuer.net', 'clk.rtpdn11.com_LOOP_1', 'avast-2015.en.softonic.com', 'radiocustoms.cdnstream1.com', 'walletwise.us', 'friends4xxx.com', '7075.roberat.com', 'search.proverka5.xyz'])):
                        continue
                    
                    # handle ww1, ww2, ...
                    if final_chain[0] == '.'.join(final_chain[1].split('.')[1:]):
                        final_chain.pop(1) 
                    
                    if final_chain[1] in merge_dic:
                        first_domain.add(final_chain[0])
                        valid_first_ips['tier_1'].append((final_chain[0], domain_ip.get(final_chain[0], None)))
                        final_chain[0] = 'PHISHING_' + merge_dic[final_chain[1]]
                    else:
                        continue
                    
                    # add remaining ip addresses
                    valid_first_ips['tier_3'].append((final_chain[-1], domain_ip.get(final_chain[-1], None)))
                    if len(final_chain) > 2:
                        valid_first_ips['tier_2'].append((final_chain[1], domain_ip.get(final_chain[1], None)))
                    
                    if '.icu' in final_chain[-1]:
                        final_chain[-1] = 'ADWARE_2'
                    if 'softonic.com' in final_chain[-1]:
                        final_chain[-1] = 'ADWARE_1'                
                
                if prefix == 'nycpost_pro':
                    mothership_ip = set(['b6b4rhsbdawj.com', 'gdl5vtbbv0lj.com', '05xz0h2xo799.com', '14nuzznszbdp.com',
                                         'fadskis.com', '8gzijqo4c85j.com', 'fadspms.com', 'es54i9g7p7xh.com'])
                    if len(set(final_chain).intersection(mothership_ip)) < 1:
                        continue
                    valid_first_ips['tier_1'].append((final_chain[0], domain_ip.get(final_chain[0], None)))
                    valid_first_ips['tier_2'].append((final_chain[1], domain_ip.get(final_chain[1], None)))
                    if len(final_chain) > 2:
                        valid_first_ips['tier_3'].append((final_chain[-1], domain_ip.get(final_chain[-1], None)))
                    
                    # for all domains in tier_1, assign tier_1
                    # for all domains in tier_3, assign tier_3
                    # for all domains in tier_2, assign tier_2
                    # for remaining domains, assign tier_na
                    # at frontend, add a mapping to map {tier_1, tier_2, tier_3, tier_na} to different colors
                
                last_domain.add(final_chain[-1])
                for index, url in enumerate(final_chain[:-1]):                    
                    if url not in nodes:
                        nodes[url] = counter
                        counter += 1
                    if final_chain[index+1] not in nodes:
                        nodes[final_chain[index+1]] = counter
                        counter += 1
                    links[(nodes[url], nodes[final_chain[index+1]])] += 1
#                 print()
                has_redirect += 1
        else:
            if len(trace['record']) == 1:
                    rchain = trace['record'][0]['record']['redirection_chain'] 
                    if len(rchain) == 1 and rchain[0]['url'] == "chrome-error://chromewebdata/":
#                         print('NO IP address')
                        no_ip += 1
                    else:
#                         pprint(rchain)
                        no_redirect += 1
            else:
                no_redirect += 1
                
    else:
        print('trace does not exist')
        print(filename)

        
# we need to direct cycles
# what algorithm is efficient?
# https://www.geeksforgeeks.org/detect-cycle-in-a-graph/
# we need to find the exact path that has cycle
from collections import defaultdict 
  
class Graph(): 
    def __init__(self,vertices): 
        self.graph = defaultdict(list) 
        self.V = vertices 
  
    def addEdge(self,u,v): 
        self.graph[u].append(v) 
  
    def isCyclicUtil(self, v, visited, recStack): 
  
        # Mark current node as visited and  
        # adds to recursion stack 
        visited[v] = True
        recStack[v] = True
  
        # Recur for all neighbours
        # if any neighbour is visited and in  
        # recStack then graph is cyclic 
        for neighbour in self.graph[v]: 
            if visited[neighbour] == False: 
                if self.isCyclicUtil(neighbour, visited, recStack) == True: 
                    print(v, neighbour)
                    return True
            elif recStack[neighbour] == True: 
                return True
  
        # The node needs to be poped from  
        # recursion stack before function ends 
        recStack[v] = False
        return False
  
    # Returns true if graph is cyclic else false 
    def isCyclic(self): 
        visited = [False] * self.V 
        recStack = [False] * self.V 
        for node in range(self.V): 
            if visited[node] == False: 
                if self.isCyclicUtil(node,visited,recStack) == True: 
                    return True
        return False

# nodes
# links
# we need a reverse dictionary
g = Graph(len(nodes)) 
for key in links:
    g.addEdge(key[0], key[1]) 

if g.isCyclic() == 1: 
    print("Graph has a cycle")
else: 
    print("Graph has no cycle")

total = no_redirect + has_redirect + no_ip
print(no_redirect, no_redirect / total, no_redirect / (no_redirect + has_redirect))
print(has_redirect, has_redirect / total, has_redirect / (no_redirect + has_redirect))
print(no_ip, no_ip / total)

print(first_domain.intersection(last_domain))
print(first_domain)
# 


36421
36421
trace does not exist
/Users/zc/Documents/TwitterBotProject/malicious_domain/64_32_8_70_TIME_2020-03-01-22-30-00//trace.json/trace.json
trace does not exist
/Users/zc/Documents/TwitterBotProject/malicious_domain/64_32_8_70_TIME_2020-03-01-22-30-00//.DS_Store/trace.json
trace does not exist
/Users/zc/Documents/TwitterBotProject/malicious_domain/64_32_8_70_TIME_2020-03-01-22-30-00//http.png/trace.json
trace does not exist
/Users/zc/Documents/TwitterBotProject/malicious_domain/64_32_8_70_TIME_2020-03-01-22-30-00//http.html/trace.json
Graph has no cycle
28169 0.866018999600332 0.8785790031813362
3893 0.1196851846158576 0.12142099681866383
465 0.014295815783810372
set()
{'dlmacsoft.com', 'tomible.com', 'zampaseries.com', 'xinh2k.com', '6068a17eed25.com', 'rickshide.com', 'public-tsokkos.com', 'omniprix.com', 'cmarcreditunion.com', 'hostdomain.com.au', 'williampichardo.com', 'garden999.com', 'youngtrapsquad808.com', 'wpzenith.com', 'oussamasat.com', 'dlttr.com', 'f2dnb9.com', 'ozf

In [154]:
# nodes_reverse = {v:k for k, v in nodes.items()}
# nums = [394,146,57,394,49,57,32,49,31,32,30,31,29,30]
# [nodes_reverse[num] for num in nums]
nodes


{'cheong.ca': 0,
 '6109.roberat.com': 1,
 'lp.advancedmactuneup.com': 2,
 'london-escort-girls.net': 3,
 'lp.smartdrivercare.com': 4,
 'giac.ca': 5,
 'liberalgovernment.ca': 6,
 'amanda.tncrun.net': 7,
 'usd.alwilda-ram.com': 8,
 'engine.trackingurl.ai': 9,
 'digitaltrends.com': 10,
 'sexdaum.net': 11,
 'hairbrushes.ca': 12,
 'xn--tauschbrsen-xfb.ch': 13,
 '3619.roberat.com': 14,
 'sectionfunction.com': 15,
 'dslhawaii.net': 16,
 'vancouvercounselling.ca': 17,
 'xn--mdel-loa.ch': 18,
 '5339.roberat.com': 19,
 'tr.trackingsys.tech': 20,
 'focus.vuer.net': 21,
 'playgreatlyoriginalfreeware.icu': 22,
 'click.bounceads.net': 23,
 'driverfixersoftware.com': 24,
 'dogscreens.ca': 25,
 'resultshunt.com': 26,
 'powerofsearches.com': 27,
 'bing.com': 28,
 'clubrusse.net': 29,
 'liberarmoviles.net': 30,
 'infopicked.com': 31,
 'p237996.infopicked.com': 32,
 'uthorner.info': 33,
 'wolve.pro': 34,
 'p8.wolve.pro': 35,
 'p7.wolve.pro': 36,
 'p6.wolve.pro': 37,
 'p5.wolve.pro': 38,
 'p4.wolve.pro': 

In [155]:
print(len(nodes))
print(len(links))
from collections import Counter
from query_viewdns import collect_ip_info_json
ips = {k: dict(Counter(v)) for k, v in valid_first_ips.items()}
filename = 'ip_domain_by_tier_{}.json'.format(prefix)

# pprint({k: Counter(v).most_common() for k, v in valid_first_ips.items()})
# collect_ip_info_json(ips, filename)

def get_low_degree_domain(ip_map):
    domain_count = Counter()
    for pair, count in ip_map['tier_3'].items():
        domain, ip = pair
        if '.softonic' not in domain and '.icu' not in domain:
            domain_count[domain] += count

    return [domain for domain, count in domain_count.most_common() if count <= 30]
                        
low_degree_domain = get_low_degree_domain(ips)
print(low_degree_domain)

# pprint(ips)
# 'finder.com', 'uk.stop-bot.com', 'sciencenaturalsupplements.com', 'fr.stop-bot.com', 'affbank.com', 'antifraud-system.com', 'secured.mobile-redirect.com', 'install.convertmyvid.com', 'driverfixersoftware.com'



638
648


KeyError: 'tier_3'

In [157]:
nodes = list(map(lambda x: {'name': x[0], 'node': x[1]}, nodes.items()))
links = list(map(lambda x: {'source': x[0][0], 'target': x[0][1], 'value': x[1]}, links.items()))

In [158]:
# assign tier 1
tier_1_domain = set([entry[0] for entry in valid_first_ips['tier_1']])
tier_2_domain = set([entry[0] for entry in valid_first_ips['tier_2']])
tier_3_domain = set([entry[0] for entry in valid_first_ips['tier_3']])

for node in nodes:
    if 'PHISHING' in node['name']:
        node['tier'] = 'tier_1' 
    elif node['name'] in tier_2_domain:
        node['tier'] = 'tier_2'
    elif 'ADWARE' in node['name'] or node['name'] in tier_3_domain:
        node['tier'] = 'tier_3'
    else:
        node['tier'] = 'other'
        
if prefix=='nycpost_pro':
    print(tier_1_domain)
    for node in nodes:
        print(node['name'])
        if node['name'] in tier_1_domain:
            node['tier'] = 'tier_1'
        elif node['name'] in tier_2_domain:
            node['tier'] = 'tier_2'
        elif node['name'] in tier_3_domain:
            node['tier'] = 'tier_3'
        else:
            node['tier'] = 'other'


In [159]:
if graph_type == 'domain':
    json.dump({'nodes': nodes, 'links': links}, open('MonitorWebApp/templates/sankey_{}.json'.format(prefix), 'w'))
else:
    json.dump({'nodes': nodes, 'links': links}, open('MonitorWebApp/templates/sankey_{}_ip.json'.format(prefix), 'w'))

In [160]:
{'nodes': nodes, 'links': links}

{'nodes': [{'name': 'cheong.ca', 'node': 0, 'tier': 'other'},
  {'name': '6109.roberat.com', 'node': 1, 'tier': 'other'},
  {'name': 'lp.advancedmactuneup.com', 'node': 2, 'tier': 'other'},
  {'name': 'london-escort-girls.net', 'node': 3, 'tier': 'other'},
  {'name': 'lp.smartdrivercare.com', 'node': 4, 'tier': 'other'},
  {'name': 'giac.ca', 'node': 5, 'tier': 'other'},
  {'name': 'liberalgovernment.ca', 'node': 6, 'tier': 'other'},
  {'name': 'amanda.tncrun.net', 'node': 7, 'tier': 'other'},
  {'name': 'usd.alwilda-ram.com', 'node': 8, 'tier': 'other'},
  {'name': 'engine.trackingurl.ai', 'node': 9, 'tier': 'other'},
  {'name': 'digitaltrends.com', 'node': 10, 'tier': 'other'},
  {'name': 'sexdaum.net', 'node': 11, 'tier': 'other'},
  {'name': 'hairbrushes.ca', 'node': 12, 'tier': 'other'},
  {'name': 'xn--tauschbrsen-xfb.ch', 'node': 13, 'tier': 'other'},
  {'name': '3619.roberat.com', 'node': 14, 'tier': 'other'},
  {'name': 'sectionfunction.com', 'node': 15, 'tier': 'other'},
  {'

NameError: name 'nodes' is not defined

In [ ]:
import requests
source = requests.get('https://raw.githubusercontent.com/drmikecrowe/mbfcext/master/docs/revised/csources.json')

In [ ]:
import pandas as pd

def save_src_to_csv(input_source):
    src = input_source.json()
    df = pd.DataFrame(src.values())
    df = df.rename(columns={"d": "domain", "b": "bias", "f": "factual_rating"})
    print(df.columns)
    df.to_csv("seed_files/mbfc_fake_list.csv")

# example use case:
# save_src_to_csv(source)

    
# print(len(src))
# for k, v in src.items():
#     if 'r' not in v:
#         pprint(v)
#         print(v['b'])
        
#     if "16" in k.lower():
#         print(v['r'], v['b'])
#     v['r'] in ['LOW']

In [ ]:
# overlap analysis


In [ ]:
pre_list = ['207_244_67_139_TIME_2020-03-01-22-28-12',
'46_166_182_113_TIME_2020-03-05-21-19-39',
'64_32_8_68_TIME_2020-03-01-22-26-19',
'181_214_86_147_TIME_2020-03-02-23-41-32',
'5_79_68_108_TIME_2020-03-01-22-05-04']
# prefix = 'mbfc'
# directory = '/Users/zc/Documents/TwitterBotProject/malicious_domain/{}/'.format(prefix)
ll = []
for prefix in pre_list:
        domains = json.load(open('reverse_lookup_json/reverse_lookup_{}.json'.format(prefix), 'r'))['response']['domains']
        domains = [d['name'] for d in domains]
        print(len(domains))
        ll.append(set(domains))
        
for i in range(len(ll)-1):
    for j in range(i+1, len(ll)):
        print(i, j, len(ll[i].intersection(ll[j])))
        print(ll[i].intersection(ll[j]))
        

In [ ]:
sources = ['mbfc', 'politifact', 'opensources', 'buzzfeed', 'allcott']

# combine 4 sources into one CSV file
from collections import defaultdict
domain_source_map = defaultdict(set)
import pandas as pd
for source in sources:
    df = pd.read_csv('seed_files/{}_fake_list.csv'.format(source))
    domains = df['domain']
    for domain in domains:
        domain_source_map[util.get_normalized_domain(domain).lower()].add(source)

# pprint(list(domain_source_map.items()))
in_pair = list(domain_source_map.items())
in_pair = [(pair[0], list(pair[1])) for pair in in_pair]
df_2 = pd.DataFrame(in_pair, columns=['domain', 'source'])
df_2.to_csv('seed_files/concat_fake_list.csv')
# df_2.to_csv('seed_files/allcott_fake_list.csv')
# save concat file to another CSV



In [ ]:

# df = pd.read_csv('~/Downloads/FB-shares---combined.csv')
# domains = [util.get_normalized_domain(url) for url in df['URL'] if type(url) == type('string')]
# len(domains)

# df_2 = pd.DataFrame(domains, columns=['domain'])
# df_2.to_csv('seed_files/allcott_fake_list.csv')


In [6]:
import json
urls = json.load(open('/Users/zc/Documents/TwitterBotProject/MonitorWebApp/templates/sankey_cnnews3_com.json', 'r'))
[node['name'] for node in urls['nodes'] if node['tier'] == 'tier_1']







['PHISHING_above', 'PHISHING_trellian']

In [24]:
import pandas as pd
df = pd.DataFrame()
df['url'] = sorted(first_domain, reverse=True)
df['campaign-id'] = ['3'] * len(first_domain)
df.to_csv('suspicious_domain_3.csv', index=False)